### Sandbox for exporting camera metadata from a database

In [ ]:
from pathlib import Path

import polars as pl

from afft.io.sql import create_endpoint
from afft.utils.log import logger
from afft.utils.result import Ok, Err

from afft.tasks.process_tables import process_database_tables


def export_table(table: pl.DataFrame, output: Path) -> None:
    """Exports a table to the given path."""
    if not output.exists():
        table.write_csv(output)
    else:
        logger.error(f"file already exists: {output}")


def main() -> None:
    """Main function."""

    queries: dict[str, str] = {
        "ctd": "SELECT * from r23685bc_20140616_225022_ctd_seabird",
        "dvl": "SELECT * from r23685bc_20140616_225022_dvl_teledyne",
        "image": "SELECT * from r23685bc_20140616_225022_image_capture",
        "pressure": "SELECT * from r23685bc_20140616_225022_pressure_parosci",
    }

    OUTPUT: Path = Path(
        "/data/kingston_snv_01/acfr_camera_metadata/r23685bc_20140616_225022_camera_metadata.csv"
    )

    match create_endpoint(database="acfr_auv_messages", host="localhost", port=5432):
        case Ok(endpoint):
            processed: pl.DataFrame = process_database_tables(
                endpoint, queries, base="image"
            )
            export_table(processed, OUTPUT)
            logger.info("Processed table successfully.")
        case Err(message):
            logger.error(message)


# INVOKE MAIN
main()